In [3]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO  # For handling HTML as a string

In [ ]:
# Directory to save the CSV files
output_dir = "league_standings"
os.makedirs(output_dir, exist_ok=True)

team_name_changes = {
    # Team Name Changes (Original -> New Name) after 1980-81 season
    "San Diego Clippers": "Los Angeles Clippers",               # 1984
    "Kansas City Kings": "Sacramento Kings",                    # 1985
    "Washington Bullets": "Washington Wizards",                 # 1997
    "Vancouver Grizzlies": "Memphis Grizzlies",                 # 2001
    "Charlotte Hornets": "New Orleans Hornets",                 # 2002
    "New Orleans–Oklahoma City Hornets": "New Orleans Hornets", # 2007
    "Seattle Supersonics": "Oklahoma City Thunder",             # 2008
    "New Jersey Nets": "Brooklyn Nets",                         # 2012
    "New Orleans Hornets": "New Orleans Pelicans",              # 2013
    "Charlotte Bobcats": "Charlotte Hornets",                   # 2014    
}

team_history_mapping = {
    # 1980-1983 - 1980 expansion and realignment
    "1980-1983": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Washington Bullets": {"conference": "Eastern", "division": "Atlantic"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Central"},
        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Midwest"},
        "Denver Nuggets": {"conference": "Western", "division": "Midwest"},
        "Houston Rockets": {"conference": "Western", "division": "Midwest"},
        "Kansas City Kings": {"conference": "Western", "division": "Midwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Midwest"},
        "Utah Jazz": {"conference": "Western", "division": "Midwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Pacific"},
        "San Diego Clippers": {"conference": "Western", "division": "Pacific"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Pacific"},
    },

    # 1984 - Clippers relocate to Los Angeles
    "1984": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Washington Bullets": {"conference": "Eastern", "division": "Atlantic"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Central"},
        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Midwest"},
        "Denver Nuggets": {"conference": "Western", "division": "Midwest"},
        "Houston Rockets": {"conference": "Western", "division": "Midwest"},
        "Kansas City Kings": {"conference": "Western", "division": "Midwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Midwest"},
        "Utah Jazz": {"conference": "Western", "division": "Midwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Pacific"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Pacific"},
    },

    # 1985-1987 - Kings relocate to Sacramento
    "1985-1987": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Washington Bullets": {"conference": "Eastern", "division": "Atlantic"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Central"},
        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Midwest"},
        "Denver Nuggets": {"conference": "Western", "division": "Midwest"},
        "Houston Rockets": {"conference": "Western", "division": "Midwest"},
        "Sacramento Kings": {"conference": "Western", "division": "Midwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Midwest"},
        "Utah Jazz": {"conference": "Western", "division": "Midwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Pacific"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Pacific"},
    },

    # 1988 - 1988 expansion
    "1988": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "Miami Heat": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Washington Bullets": {"conference": "Eastern", "division": "Atlantic"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Central"},
        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},
    
        
        "Dallas Mavericks": {"conference": "Western", "division": "Midwest"},
        "Denver Nuggets": {"conference": "Western", "division": "Midwest"},
        "Houston Rockets": {"conference": "Western", "division": "Midwest"},
        "Charlotte Hornets": {"conference": "Western", "division": "Midwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Midwest"},
        "Utah Jazz": {"conference": "Western", "division": "Midwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Pacific"},
    },
                                                              
    # 1989 - 1989 expansion
    "1989": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "Miami Heat": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Washington Bullets": {"conference": "Eastern", "division": "Atlantic"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Central"},
        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},
        "Orlando Magic": {"conference": "Eastern", "division": "Central"},
        
        "Charlotte Hornets": {"conference": "Western", "division": "Midwest"},
        "Dallas Mavericks": {"conference": "Western", "division": "Midwest"},
        "Denver Nuggets": {"conference": "Western", "division": "Midwest"},
        "Houston Rockets": {"conference": "Western", "division": "Midwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Midwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Midwest"},
        "Utah Jazz": {"conference": "Western", "division": "Midwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Pacific"},
    },

    # 1990 - Hornets and Magic switch conferences
    "1990": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "Miami Heat": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Washington Bullets": {"conference": "Eastern", "division": "Atlantic"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Central"},
        "Charlotte Hornets": {"conference": "Eastern", "division": "Central"},
        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Midwest"},
        "Denver Nuggets": {"conference": "Western", "division": "Midwest"},
        "Houston Rockets": {"conference": "Western", "division": "Midwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Midwest"},
        "Orlando Magic": {"conference": "Western", "division": "Midwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Midwest"},
        "Utah Jazz": {"conference": "Western", "division": "Midwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Pacific"},
    },
      
    # 1991-1994 - Magic moves back to the Eastern Conference
    "1991-1994": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "Miami Heat": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Orlando Magic": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Washington Bullets": {"conference": "Eastern", "division": "Atlantic"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Central"},
        "Charlotte Hornets": {"conference": "Eastern", "division": "Central"},
        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Midwest"},
        "Denver Nuggets": {"conference": "Western", "division": "Midwest"},
        "Houston Rockets": {"conference": "Western", "division": "Midwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Midwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Midwest"},
        "Utah Jazz": {"conference": "Western", "division": "Midwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Pacific"},
    },

    # 1995-1996 - 1995 expansion
    "1995-1996": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "Miami Heat": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Orlando Magic": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Washington Bullets": {"conference": "Eastern", "division": "Atlantic"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Central"},
        "Charlotte Hornets": {"conference": "Eastern", "division": "Central"},
        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},
        "Toronto Raptors": {"conference": "Eastern", "division": "Central"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Midwest"},
        "Denver Nuggets": {"conference": "Western", "division": "Midwest"},
        "Houston Rockets": {"conference": "Western", "division": "Midwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Midwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Midwest"},
        "Utah Jazz": {"conference": "Western", "division": "Midwest"},
        "Vancouver Grizzlies": {"conference": "Western", "division": "Midwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Pacific"},
    },
    
    # 1997-2000 - Washington renaming
    "1997-2000": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "Miami Heat": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Orlando Magic": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Washington Wizards": {"conference": "Eastern", "division": "Atlantic"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Central"},
        "Charlotte Hornets": {"conference": "Eastern", "division": "Central"},
        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},
        "Toronto Raptors": {"conference": "Eastern", "division": "Central"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Midwest"},
        "Denver Nuggets": {"conference": "Western", "division": "Midwest"},
        "Houston Rockets": {"conference": "Western", "division": "Midwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Midwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Midwest"},
        "Utah Jazz": {"conference": "Western", "division": "Midwest"},
        "Vancouver Grizzlies": {"conference": "Western", "division": "Midwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Pacific"},
    },

    # 2001 - Grizzlies relocate to Memphis
    "2001": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "Miami Heat": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Orlando Magic": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Washington Wizards": {"conference": "Eastern", "division": "Atlantic"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Central"},
        "Charlotte Hornets": {"conference": "Eastern", "division": "Central"},
        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},
        "Toronto Raptors": {"conference": "Eastern", "division": "Central"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Midwest"},
        "Denver Nuggets": {"conference": "Western", "division": "Midwest"},
        "Houston Rockets": {"conference": "Western", "division": "Midwest"},
        "Memphis Grizzlies": {"conference": "Western", "division": "Midwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Midwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Midwest"},
        "Utah Jazz": {"conference": "Western", "division": "Midwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Pacific"},
    },

    # 2002-2003 - Hornets relocate to New Orleans
    "2002-2003": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "Miami Heat": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Orlando Magic": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Washington Wizards": {"conference": "Eastern", "division": "Atlantic"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Central"},
        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},
        "Toronto Raptors": {"conference": "Eastern", "division": "Central"},
        "New Orleans Hornets": {"conference": "Eastern", "division": "Central"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Midwest"},
        "Denver Nuggets": {"conference": "Western", "division": "Midwest"},
        "Houston Rockets": {"conference": "Western", "division": "Midwest"},
        "Memphis Grizzlies": {"conference": "Western", "division": "Midwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Midwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Midwest"},
        "Utah Jazz": {"conference": "Western", "division": "Midwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Pacific"},
    },

    # 2004 - 2004 expansion and realignment
    "2004": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Toronto Raptors": {"conference": "Eastern", "division": "Atlantic"},

        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Southeast"},
        "Charlotte Bobcats": {"conference": "Eastern", "division": "Southeast"},
        "Miami Heat": {"conference": "Eastern", "division": "Southeast"},
        "Orlando Magic": {"conference": "Eastern", "division": "Southeast"},
        "Washington Wizards": {"conference": "Eastern", "division": "Southeast"},
        
        "Denver Nuggets": {"conference": "Western", "division": "Northwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Northwest"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Northwest"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Northwest"},
        "Utah Jazz": {"conference": "Western", "division": "Northwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Southwest"},
        "Houston Rockets": {"conference": "Western", "division": "Southwest"},
        "Memphis Grizzlies": {"conference": "Western", "division": "Southwest"},
        "New Orleans Hornets": {"conference": "Western", "division": "Southwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Southwest"},
    },

    # 2005-2005 - Hurricane Katrina impacts Hornet
    "2005-2006": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Toronto Raptors": {"conference": "Eastern", "division": "Atlantic"},

        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Southeast"},
        "Charlotte Bobcats": {"conference": "Eastern", "division": "Southeast"},
        "Miami Heat": {"conference": "Eastern", "division": "Southeast"},
        "Orlando Magic": {"conference": "Eastern", "division": "Southeast"},
        "Washington Wizards": {"conference": "Eastern", "division": "Southeast"},
        
        "Denver Nuggets": {"conference": "Western", "division": "Northwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Northwest"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Northwest"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Northwest"},
        "Utah Jazz": {"conference": "Western", "division": "Northwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Southwest"},
        "Houston Rockets": {"conference": "Western", "division": "Southwest"},
        "Memphis Grizzlies": {"conference": "Western", "division": "Southwest"},
        "New Orleans–Oklahoma City Hornets": {"conference": "Western", "division": "Southwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Southwest"},
    },

    # 2007 - Hornets return to New Orleans full-time
    "2007": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Toronto Raptors": {"conference": "Eastern", "division": "Atlantic"},

        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Southeast"},
        "Charlotte Bobcats": {"conference": "Eastern", "division": "Southeast"},
        "Miami Heat": {"conference": "Eastern", "division": "Southeast"},
        "Orlando Magic": {"conference": "Eastern", "division": "Southeast"},
        "Washington Wizards": {"conference": "Eastern", "division": "Southeast"},
        
        "Denver Nuggets": {"conference": "Western", "division": "Northwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Northwest"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Northwest"},
        "Seattle SuperSonics": {"conference": "Western", "division": "Northwest"},
        "Utah Jazz": {"conference": "Western", "division": "Northwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Southwest"},
        "Houston Rockets": {"conference": "Western", "division": "Southwest"},
        "Memphis Grizzlies": {"conference": "Western", "division": "Southwest"},
        "New Orleans Hornets": {"conference": "Western", "division": "Southwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Southwest"},
    },

    # 2008-2011 - SuperSonics relocate to Oklahoma City
    "2008-2011": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "New Jersey Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Toronto Raptors": {"conference": "Eastern", "division": "Atlantic"},

        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Southeast"},
        "Charlotte Bobcats": {"conference": "Eastern", "division": "Southeast"},
        "Miami Heat": {"conference": "Eastern", "division": "Southeast"},
        "Orlando Magic": {"conference": "Eastern", "division": "Southeast"},
        "Washington Wizards": {"conference": "Eastern", "division": "Southeast"},
        
        "Denver Nuggets": {"conference": "Western", "division": "Northwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Northwest"},
        "Oklahoma City Thunder": {"conference": "Western", "division": "Northwest"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Northwest"},
        "Utah Jazz": {"conference": "Western", "division": "Northwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Southwest"},
        "Houston Rockets": {"conference": "Western", "division": "Southwest"},
        "Memphis Grizzlies": {"conference": "Western", "division": "Southwest"},
        "New Orleans Hornets": {"conference": "Western", "division": "Southwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Southwest"},
    },
    
    # 2012 - Nets relocate to Brooklyn
    "2012": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "Brooklyn Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Toronto Raptors": {"conference": "Eastern", "division": "Atlantic"},

        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Southeast"},
        "Charlotte Bobcats": {"conference": "Eastern", "division": "Southeast"},
        "Miami Heat": {"conference": "Eastern", "division": "Southeast"},
        "Orlando Magic": {"conference": "Eastern", "division": "Southeast"},
        "Washington Wizards": {"conference": "Eastern", "division": "Southeast"},
        
        "Denver Nuggets": {"conference": "Western", "division": "Northwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Northwest"},
        "Oklahoma City Thunder": {"conference": "Western", "division": "Northwest"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Northwest"},
        "Utah Jazz": {"conference": "Western", "division": "Northwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Southwest"},
        "Houston Rockets": {"conference": "Western", "division": "Southwest"},
        "Memphis Grizzlies": {"conference": "Western", "division": "Southwest"},
        "New Orleans Hornets": {"conference": "Western", "division": "Southwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Southwest"},
    },

    # 2013 - New Orleans renaming
    "2013": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "Brooklyn Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Toronto Raptors": {"conference": "Eastern", "division": "Atlantic"},

        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Southeast"},
        "Charlotte Bobcats": {"conference": "Eastern", "division": "Southeast"},
        "Miami Heat": {"conference": "Eastern", "division": "Southeast"},
        "Orlando Magic": {"conference": "Eastern", "division": "Southeast"},
        "Washington Wizards": {"conference": "Eastern", "division": "Southeast"},
        
        "Denver Nuggets": {"conference": "Western", "division": "Northwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Northwest"},
        "Oklahoma City Thunder": {"conference": "Western", "division": "Northwest"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Northwest"},
        "Utah Jazz": {"conference": "Western", "division": "Northwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Southwest"},
        "Houston Rockets": {"conference": "Western", "division": "Southwest"},
        "Memphis Grizzlies": {"conference": "Western", "division": "Southwest"},
        "New Orleans Pelicans": {"conference": "Western", "division": "Southwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Southwest"},
    },
    
    # 2014-2024 - Charlotte renaming
    "2014-2024": {
        "Boston Celtics": {"conference": "Eastern", "division": "Atlantic"},
        "Brooklyn Nets": {"conference": "Eastern", "division": "Atlantic"},
        "New York Knicks": {"conference": "Eastern", "division": "Atlantic"},
        "Philadelphia 76ers": {"conference": "Eastern", "division": "Atlantic"},
        "Toronto Raptors": {"conference": "Eastern", "division": "Atlantic"},

        "Chicago Bulls": {"conference": "Eastern", "division": "Central"},
        "Cleveland Cavaliers": {"conference": "Eastern", "division": "Central"},
        "Detroit Pistons": {"conference": "Eastern", "division": "Central"},
        "Indiana Pacers": {"conference": "Eastern", "division": "Central"},
        "Milwaukee Bucks": {"conference": "Eastern", "division": "Central"},

        "Atlanta Hawks": {"conference": "Eastern", "division": "Southeast"},
        "Charlotte Hornets": {"conference": "Eastern", "division": "Southeast"},
        "Miami Heat": {"conference": "Eastern", "division": "Southeast"},
        "Orlando Magic": {"conference": "Eastern", "division": "Southeast"},
        "Washington Wizards": {"conference": "Eastern", "division": "Southeast"},
        
        "Denver Nuggets": {"conference": "Western", "division": "Northwest"},
        "Minnesota Timberwolves": {"conference": "Western", "division": "Northwest"},
        "Oklahoma City Thunder": {"conference": "Western", "division": "Northwest"},
        "Portland Trail Blazers": {"conference": "Western", "division": "Northwest"},
        "Utah Jazz": {"conference": "Western", "division": "Northwest"},

        "Golden State Warriors": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Clippers": {"conference": "Western", "division": "Pacific"},
        "Los Angeles Lakers": {"conference": "Western", "division": "Pacific"},
        "Phoenix Suns": {"conference": "Western", "division": "Pacific"},
        "Sacramento Kings": {"conference": "Western", "division": "Pacific"},
        
        "Dallas Mavericks": {"conference": "Western", "division": "Southwest"},
        "Houston Rockets": {"conference": "Western", "division": "Southwest"},
        "Memphis Grizzlies": {"conference": "Western", "division": "Southwest"},
        "New Orleans Pelicans": {"conference": "Western", "division": "Southwest"},
        "San Antonio Spurs": {"conference": "Western", "division": "Southwest"},
    },
}

# Function to get the correct season range from the year
def get_season_range(year):
    if 1980 <= year <= 1983:
        return "1980-1983"
    elif year == 1984:
        return "1984"
    elif 1985 <= year <= 1987:
        return "1985-1987"
    elif year == 1988:
        return "1988"
    elif year == 1989:
        return "1989"
    elif year == 1990:
        return "1990"
    elif 1991 <= year <= 1994:
        return "1991-1994"
    elif 1995 <= year <= 1996:
        return "1995-1996"
    elif 1997 <= year <= 2000:
        return "1997-2000"
    elif year == 2001:
        return "2001"
    elif 2002 <= year <= 2003:
        return "2002-2003"
    elif year == 2004:
        return "2004"
    elif 2005 <= year <= 2006:
        return "2005-2006"
    elif year == 2007:
        return "2007"
    elif 2008 <= year <= 2011:
        return "2008-2011"
    elif year == 2012:
        return "2012"
    elif year == 2013:
        return "2013"
    elif 2014 <= year <= 2024:
        return "2014-2024"
    else:
        return None

# Function to handle team name changes
def normalize_team_name(team_name):
    normalized_name = team_name_changes.get(team_name, team_name)
    #print(f"Original: {team_name}, Normalized: {normalized_name}")  # Debugging line
    return normalized_name

# Function to scrape and save standings for a specific season
def scrape_season_standings(season_start_year):
    season_end_year = season_start_year + 1
    season = f"{season_start_year}-{str(season_end_year)[-2:]}"
    
    # URL for basketball-reference standings
    url = f"https://www.basketball-reference.com/leagues/NBA_{season_end_year}_standings.html"
    
    try:
        # Fetch the webpage
        response = requests.get(url)
        response.raise_for_status()
        
        # Parse the HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract all tables from the page
        html_content = str(soup)
        tables = pd.read_html(StringIO(html_content))  # Use StringIO to read HTML from a string
        
        # Check how many tables were found and print them for debugging
        print(f"Found {len(tables)} tables for season {season}")
        
        # If no tables are found, return
        if not tables:
            print(f"No tables found for {season}.")
            return
        
        # Assume the first two tables are Eastern and Western standings
        eastern_standings = tables[0].copy()  # Make a copy to avoid SettingWithCopyWarning
        western_standings = tables[1].copy()  # Make a copy to avoid SettingWithCopyWarning
        
        # Rename 'Eastern Conference' or 'Western Conference' to 'Team'
        eastern_standings.rename(columns={eastern_standings.columns[0]: 'Team'}, inplace=True)
        western_standings.rename(columns={western_standings.columns[0]: 'Team'}, inplace=True)
        
        # Remove rows containing division names like "Atlantic Division"
        eastern_standings = eastern_standings[~eastern_standings['Team'].str.contains('Division')]
        western_standings = western_standings[~western_standings['Team'].str.contains('Division')]
        
        # Remove '*' from team names before normalizing
        eastern_standings['Team'] = eastern_standings['Team'].str.replace('*', '', regex=False)
        western_standings['Team'] = western_standings['Team'].str.replace('*', '', regex=False)
        
        # Add a new column 'Normalized Team' with the normalized team names
        eastern_standings['Normalized Team'] = eastern_standings['Team'].apply(normalize_team_name)
        western_standings['Normalized Team'] = western_standings['Team'].apply(normalize_team_name)
        
        # Add 'Conference' column for both standings
        eastern_standings['Conference'] = 'Eastern'
        western_standings['Conference'] = 'Western'
        
        # Get the correct season range based on the season year
        season_range = get_season_range(season_start_year)
        if not season_range:
            print(f"No mapping found for season {season}")
            return
        
        # Retrieve division information from the team history mapping
        season_division_mapping = team_history_mapping.get(season_range, {})
        
        # Debugging: Print out the season division mapping
        # print(f"Season: {season}")
        # print(f"Season Range: {season_range}")
        # print(f"Season Division Mapping: {season_division_mapping}")
        
        # Extract division information for each team in the standings
        eastern_standings['Division'] = eastern_standings['Team'].map(
            lambda team: season_division_mapping.get(team, {}).get('division', 'Unknown')
        )
        western_standings['Division'] = western_standings['Team'].map(
            lambda team: season_division_mapping.get(team, {}).get('division', 'Unknown')
        )
        
        # Combine the two tables (Eastern and Western)
        combined_standings = pd.concat([eastern_standings, western_standings], ignore_index=True)
        
        # Save to CSV
        output_path = os.path.join(output_dir, f"{season}_league_standings.csv")
        combined_standings.to_csv(output_path, index=False)
        print(f"Saved: {output_path}")
    
    except Exception as e:
        print(f"Failed to scrape standings for {season}: {e}")
        raise  # Reraise the exception to debug further

# Loop through all years from 1981 to the present
#for year in range(1981, 2025):  # Adjust to the current year
#    scrape_season_standings(year)

#scrape_season_standings(1980)


Found 2 tables for season 2011-12
Saved: league_standings/2011-12_league_standings.csv
Found 2 tables for season 2012-13
Saved: league_standings/2012-13_league_standings.csv
Found 2 tables for season 2013-14
Saved: league_standings/2013-14_league_standings.csv
Found 2 tables for season 2014-15
Saved: league_standings/2014-15_league_standings.csv
Found 4 tables for season 2015-16
Saved: league_standings/2015-16_league_standings.csv
Found 4 tables for season 2016-17
Saved: league_standings/2016-17_league_standings.csv
Found 4 tables for season 2017-18
Saved: league_standings/2017-18_league_standings.csv
Found 4 tables for season 2018-19
Saved: league_standings/2018-19_league_standings.csv
Found 4 tables for season 2019-20
Saved: league_standings/2019-20_league_standings.csv
Found 4 tables for season 2020-21
Saved: league_standings/2020-21_league_standings.csv
Found 4 tables for season 2021-22
Saved: league_standings/2021-22_league_standings.csv
Found 4 tables for season 2022-23
Saved: le